In [3]:
import numpy as np
import edhec_risk_kit_206 as erk

ind49_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":]
ind49_mcap = erk.get_ind_market_caps(49, weights=True)["2013":]
inds = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']
rho_ = ind49_rets[inds].corr()
vols_ = (ind49_rets[inds].std()*np.sqrt(12))    
sigma_prior_ =  (vols_.T).dot(vols_) * rho_

In [26]:
import numpy as np
import pandas as pd

def as_colvec(x):
    if (x.ndim == 2):
        return x
    else:
        return np.expand_dims(x, axis=1)

def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)


In [20]:
w_eq = ind49_mcap[inds].iloc[0]
w_eq = w_eq/w_eq.sum()

In [25]:
w_eq.sort_values()

Hlth     0.041663
Whlsl    0.097411
Food     0.139176
Fin      0.175362
Rtail    0.546388
Name: 2013-01, dtype: float64

In [24]:
implied_returns(2.5,sigma_prior_,w_eq).sort_values()

Hlth     0.152910
Food     0.158115
Fin      0.175580
Whlsl    0.201836
Rtail    0.224827
Name: Implied Returns, dtype: float64

In [42]:
# q = pd.Series([0.03])
q = pd.Series([0.05])


p = pd.DataFrame([0.]*len(inds),index=inds).T
w_rtail = w_eq.loc['Rtail']/(w_eq.loc['Rtail']+w_eq.loc['Whlsl'])
w_whlsl = w_eq.loc['Whlsl']/(w_eq.loc['Rtail']+w_eq.loc['Whlsl'])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_rtail
p.iloc[0]['Whlsl'] = -w_whlsl

In [43]:
-w_whlsl,-w_rtail.round(2)

(-0.15130613851608934, -0.85)

In [44]:
delta = 2.5
tau = 0.05
bl_mu, bl_sigma = bl(w_eq, sigma_prior_, p, q, tau = tau)
bl_mu.round(2).sort_values(ascending=False)

Hlth     0.19
Whlsl    0.19
Rtail    0.19
Fin      0.17
Food     0.14
dtype: float64

In [46]:
implied_returns(2.5,bl_sigma,bl_mu).sort_values(ascending=False)

Whlsl    0.189645
Rtail    0.178307
Fin      0.165185
Hlth     0.160882
Food     0.138245
Name: Implied Returns, dtype: float64

In [40]:
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)
    
def w_star(delta, sigma, mu):
    return (inverse(sigma).dot(mu))/delta

wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)

In [41]:
wstar.sort_values(ascending=False)

Hlth     0.310774
Rtail    0.290293
Fin      0.167011
Food     0.132549
Whlsl    0.051754
dtype: float64